In [ ]:
# default_exp dataset.image_object_detection

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import sys
import argparse
import logging
from os.path import join, isfile, dirname, normpath, basename
from functools import partial
from datetime import datetime
from mlcore.dataset.core import Dataset, input_feedback, configure_logging
from mlcore.dataset.type import DatasetType
from mlcore.image.pillow_tools import assign_exif_orientation
from mlcore.annotation.core import RegionShape, convert_region
from mlcore.annotation.via_adapter import read_annotations as via_read_annotations
from mlcore.annotation.via_adapter import write_annotations as via_write_annotations
from mlcore.tensorflow.tfrecord_builder import create_tfrecord_file
from mlcore.evaluation.core import box_area, intersection_box, union_box

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

CATEGORY_LABEL_KEY = 'category'
DEFAULT_CATEGORIES_FILE = 'categories.txt'
DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE = 'annotations.csv'
DEFAULT_SEGMENTATION_ANNOTATIONS_FILE = 'via_region_data.json'
DEFAULT_SPLIT = 0.2
DATA_SET_FOLDER = 'datasets'
SEMANTIC_MASK_FOLDER = 'semantic_masks'
TRAIN_VAL_FOLDER = 'trainval'
TRAIN_FOLDER = 'train'
VAL_FOLDER = 'val'
TEST_FOLDER = 'test'
NOT_CATEGORIZED = '[NOT_CATEGORIZED]'

DATASET_TYPE = DatasetType.IMAGE_OBJECT_DETECTION

In [ ]:
# export

logger = logging.getLogger(__name__)

# Dataset for image object detection

> Creates a dataset for image object detection.

Creating a dataset for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The data-set is created based on an image-set.

## Image-Set

Image-sets are collected images to build a data-set from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[image_set_type]*/*[image_set_name]*

Inside the `[image_set_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the image-set contains

## Data-Set Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[data_set_type]*/*[data_set_name]*
where `[data_set_type]` is the same as the corresponding `[image_set_type]` and `[data_set_name]` is the same as the corresponding `[image_set_name]`.

Inside the `[data_set_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the data-set contains

## Create a object detection data-set

Object detection data-set can be created from a segmentation or object-detection image-set.
All images are validated against the annotations, if they contain at least one annotation and that the annotation category belongs to one of the given categories. The annotations have to be in [VIA v1](http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.5.html) json format. Polygon annotations are converted into rectangle annotations for unique bounding-box generation.

In [ ]:
# export


class ImageObjectDetectionDataset(Dataset):
    """
    Object detection dataset.
    `name`: The name of the dataset.
    `base_path`: The data-set base-path.
    `imageset_path`: The imageset source path.
    `categories_path`: The path to the categories.txt file.
    `annotations_path`: The path to the annotations-file.
    `create_tfrecord`: Also create .tfrecord files.
    `join_overlapping_regions`: Whether overlapping regions of same category should be joined.
    `annotation_area_threshold`: Keep only annotations with minimum size (width or height) related to image size
    """

    def __init__(self, name, base_path, imageset_path, categories_path, annotations_path=None,
                 create_tfrecord=False, join_overlapping_regions=False, annotation_area_threshold=None):
        super().__init__(name, base_path, imageset_path, categories_path, DATASET_TYPE, create_tfrecord)
        self.annotations_path = annotations_path
        self.annotations = via_read_annotations(annotations_path, self.train_val_folder, CATEGORY_LABEL_KEY) if annotations_path else {}
        self.join_overlapping_regions = join_overlapping_regions
        self.annotation_area_threshold = annotation_area_threshold

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)

        # write the split train annotations
        if annotations_train:
            annotations_target_path = join(self.train_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            via_write_annotations(annotations_target_path, annotations_train, CATEGORY_LABEL_KEY)
            # if creating a .tfrecord
            if self.create_tfrecord:
                tfrecord_file_name = '{}.record'.format(normpath(basename(self.train_folder)))
                tfrecord_output_file = join(self.folder, tfrecord_file_name)
                self.logger.info('Generate file {} to {}'.format(tfrecord_file_name, self.folder))
                create_tfrecord_file(tfrecord_output_file, self.categories, annotations_train)

        # write the split val annotations
        if annotations_val:
            annotations_target_path = join(self.val_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            via_write_annotations(annotations_target_path, annotations_val, CATEGORY_LABEL_KEY)
            # if creating a .tfrecord
            if self.create_tfrecord:
                tfrecord_file_name = '{}.record'.format(normpath(basename(self.val_folder)))
                tfrecord_output_file = join(self.folder, tfrecord_file_name)
                self.logger.info('Generate file {} to {}'.format(tfrecord_file_name, self.folder))
                create_tfrecord_file(tfrecord_output_file, self.categories, annotations_val)

        return annotations_train, annotations_val

    def convert_annotations(self):
        """
        Converts segmentation regions from polygon to rectangle, if exist
        """

        # only the trainval images have annotation, not the test images
        area_threshold = self.annotation_area_threshold

        steps = [
            {
                'name': 'position',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    't': 'Trim',  # transform the annotation
                    'T': 'Trim All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: p_min < 0 or p_max >= size,
                'message': '{} -> {} : {}Exceeds image {}. \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}',
                'transform': lambda p, size=0: max(min(p, size - 1), 0),
            },
            {
                'name': 'size',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, _: p_max - p_min <= 1,
                'message': '{} -> {} : {}Shape {} is <= 1 pixel. \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}',
                'transform': lambda p, size=0: p,
            },
            {
                'name': 'area',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: area_threshold and (p_max - p_min) / size <= area_threshold,
                'message': '{} <= {} percent. {}'.format('{} -> {} : {}Shape {} is', (area_threshold or 0) * 100, ' \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}'),
                'transform': lambda p, size=0: p,
            }
        ]

        self.logger.info('Start convert image annotations from {}'.format(self.annotations_path))

        for annotation in self.annotations.values():
            # skip file, if regions are empty or file do not exist
            if not (annotation.regions and isfile(annotation.file_path)):
                continue

            # convert from polygon to rect if needed
            for region in annotation.regions:
                convert_region(region, RegionShape.RECTANGLE)

            # try to join regions
            if self.join_overlapping_regions:
                self.join_regions(annotation.regions)

            image, _, __ = assign_exif_orientation(annotation.file_path)
            img_width, img_height = image.size
            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                # validate the shape size
                x_min, x_max = region.points_x[:2]
                y_min, y_max = region.points_y[:2]
                for step in steps:
                    width_condition = step['condition'](x_min, x_max, img_width)
                    height_condition = step['condition'](y_min, y_max, img_height)
                    if width_condition or height_condition:
                        size_message = ['width'] if width_condition else []
                        size_message.extend(['height'] if height_condition else [])
                        message = step['message'].format(annotation.file_name, index, ' ',
                                                         ' and '.join(size_message),
                                                         x_min, y_min, x_max, y_max)

                        step['choice'] = input_feedback(message, step['choice'], step['choices'])

                        choice_op = step['choice'].lower()
                        # if skip the shapes
                        if choice_op == 's':
                            delete_regions[index] = True
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             x_min, y_min, x_max, y_max)
                            self.logger.info(message)

                            break
                        else:
                            region.points_x = list(map(partial(step['transform'], size=img_width), [x_min, x_max]))
                            region.points_y = list(map(partial(step['transform'], size=img_height), [y_min, y_max]))
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             x_min, y_min, x_max, y_max)
                            self.logger.info(message)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

        self.logger.info('Finished convert image annotations from {}'.format(self.annotations_path))

    def join_regions(self, regions):
        """
        Join regions which overlaps.
        `regions`: the region to parse
        """
        len_before = len(regions)
        index_left = 0
        while index_left < len(regions):
            regions_joined = []
            region_left = regions[index_left]
            for index_right in range(len(regions)):
                if index_left == index_right:
                    continue
                region_right = regions[index_right]
                same_label_length = len(region_left.labels) == len(region_right.labels)
                same_labels = same_label_length and len(region_left.labels) == len(set(region_left.labels) & set(region_right.labels))
                if same_labels:
                    bbox_left = (region_left.points_x, region_left.points_y)
                    bbox_right = (region_right.points_x, region_right.points_y)
                    inter_area = box_area(intersection_box(bbox_left, bbox_right))
                    if inter_area > 0:
                        points_x, points_y = union_box(bbox_left, bbox_right)
                        region_left.points_x = points_x
                        region_left.points_y = points_y
                        regions_joined.append(index_right)
            for index in regions_joined[::-1]:
                del regions[index]
            if not regions_joined:
                index_left += 1
        self.logger.info('Joined overlapping regions from {} -> {}.'.format(len_before, len(regions)))

## Build a data-set

To build a data-set from an image-set. Handles currently classification and segmentation image-sets taken from the image-set-type, which is the parent folder, the image-set folder is located in. 

In [ ]:
# export


def build_dataset(category_file_path, output, annotation_file_path=None, split=DEFAULT_SPLIT, seed=None, sample=0,
                  create_tfrecord=False, join_overlapping_regions=False, annotation_area_threshold=None,
                  dataset_name=None):
    """
    Build the dataset for training, Validation and test
    `category_file_path`: the filename of the categories file
    `output`: the dataset base folder to build the dataset in
    `annotation_file_path`: the file path to the annotation file
    `split`: the size of the validation set as percentage
    `seed`: random seed to reproduce splits
    `sample`: the size of the sample set as percentage
    `create_tfrecord`: Also create .tfrecord files.
    `join_overlapping_regions`: Whether overlapping regions of same category should be joined.
    `annotation_area_threshold`: Keep only annotations with minimum size (width or height) related to image size
    `dataset_name`: the name of the dataset, if not set infer from the category file path
    """
    log_memory_handler = configure_logging()

    path = dirname(category_file_path)

    # try to infer the data-set name if not explicitly set
    if dataset_name is None:
        dataset_name = basename(path)

    logger.info('Build parameters:')
    logger.info(' '.join(sys.argv[1:]))
    logger.info('Build configuration:')
    logger.info('category_file_path: {}'.format(category_file_path))
    logger.info('annotation_file_path: {}'.format(annotation_file_path))
    logger.info('split: {}'.format(split))
    logger.info('seed: {}'.format(seed))
    logger.info('sample: {}'.format(sample))
    logger.info('dataset_type: {}'.format(DATASET_TYPE))
    logger.info('output: {}'.format(output))
    logger.info('join_overlapping_regions: {}'.format(join_overlapping_regions))
    logger.info('annotation_area_threshold: {}'.format(annotation_area_threshold))
    logger.info('name: {}'.format(dataset_name))

    logger.info('Start build {} data-set {} at {}'.format(DATASET_TYPE, dataset_name, output))

    dataset = ImageObjectDetectionDataset(dataset_name, output, path, category_file_path, annotation_file_path,
                                          create_tfrecord, join_overlapping_regions, annotation_area_threshold)

    # create the dataset folders
    logger.info("Start create the dataset folders at {}".format(dataset.base_path))
    dataset.create_folders()
    logger.info("Finished create the dataset folders at {}".format(dataset.base_path))

    # create the build log file
    log_file_name = datetime.now().strftime("build_%Y.%m.%d-%H.%M.%S.log")
    file_handler = logging.FileHandler(join(dataset.folder, log_file_name), encoding="utf-8")
    log_memory_handler.setTarget(file_handler)

    # build the dataset
    dataset.build(split, seed, sample)

    logger.info('Finished build {} dataset {} at {}'.format(DATASET_TYPE, dataset_name, output))

## Run from command line

To run the dataset builder from command line, use the following command:
`python -m mlcore.dataset.image_object_detection [parameters]`

The following parameters are supported:
- `[categories]`: The path to the categories file. (e.g.: *categories.txt*)
- `--annotation`: The path to the image-set annotation file, the data-set is build from. (e.g.: *imagesets/classification/car_damage/annotations.csv* for classification, *imagesets/segmentation/car_damage/via_region_data.json* for segmentation)
- `--split`: The percentage of the data which belongs to validation set, default to *0.2* (=20%)
- `--seed`: A random seed to reproduce splits, default to None
- `--category-label-key`: The key, the category name can be found in the annotation file, default to *category*.
- `--sample`: The percentage of the data which will be copied as a sample set with in a separate folder with "_sample" suffix. If not set, no sample data-set will be created.
- `--tfrecord`: Also create .tfrecord files.
- `--join-overlapping-regions`: Whether overlapping regions of same category should be joined.
- `--annotation-area-thresh`: Keep only annotations with minimum size (width or height) related to image size.
- `--output`: The path of the dataset folder, default to *../datasets*.
- `--name`: The name of the data-set, if not explicitly set try to infer from categories file path.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("categories",
                        help="The path to the image-set categories file.")
    parser.add_argument("--annotation",
                        help="The path to the image-set annotation file, the data-set is build from.",
                        default=None)
    parser.add_argument("--split",
                        help="Percentage of the data which belongs to validation set.",
                        type=float,
                        default=0.2)
    parser.add_argument("--seed",
                        help="A random seed to reproduce splits.",
                        type=int,
                        default=None)
    parser.add_argument("--category-label-key",
                        help="The key of the category name.",
                        default=CATEGORY_LABEL_KEY)
    parser.add_argument("--sample",
                        help="Percentage of the data which will be copied as a sample set.",
                        type=float,
                        default=0)
    parser.add_argument("--tfrecord",
                        help="Also create .tfrecord files.",
                        action="store_true")
    parser.add_argument("--join-overlapping-regions",
                        help="Whether overlapping regions of same category should be joined.",
                        action="store_true")
    parser.add_argument("--annotation-area-thresh",
                        help="Keep only annotations with minimum size (width or height) related to image size.",
                        type=float,
                        default=None)
    parser.add_argument("--output",
                        help="The path of the data-set folder.",
                        default=DATA_SET_FOLDER)
    parser.add_argument("--name",
                        help="The name of the data-set, if not explicitly set try to infer from categories file path.",
                        default=None)
    args = parser.parse_args()

    CATEGORY_LABEL_KEY = args.category_label_key

    build_dataset(args.categories, args.output, args.annotation, args.split, args.seed, args.sample, args.tfrecord,
                   args.join_overlapping_regions, args.annotation_area_thresh, args.name)
